# BOW (Bag of words)

    - local word representation이면서 count-based representation
    - 단어의 등장 순서를 고려하지 않고 빈도(frequency)에 집중

In [3]:
from konlpy.tag import Okt

In [4]:
okt = Okt()

In [9]:
def bag_of_words(document):
    document = document.replace('.','')
    tokenized_document = okt.morphs(document)
    
    word_to_idx = {}
    bow = []
    
    for word in tokenized_document:
        if word not in word_to_idx.keys():
            word_to_idx[word] = len(word_to_idx)
            bow.insert(len(word_to_idx)-1, 1)

        else:
            idx = word_to_idx.get(word)
            bow[idx] = bow[idx]+1
            
    return word_to_idx, bow

In [10]:
doc1 = '정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.'
vocab, bow = bag_of_words(doc1)
print(vocab)
print(bow)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [11]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

vocab, bow = bag_of_words(doc2)
print(vocab)
print(bow)

{'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [12]:
doc3 = doc1+ ' ' + doc2

vocab, bow = bag_of_words(doc3)
print(vocab)
print(bow)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
[1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


    - doc3은 doc1, doc2 의 단어들을 모두 포함하고 있음
    - bow는 9여러 문서의 단어 집합을 합친 뒤 해당 단어 집합에 대한 각 문서의 bow를 구하기도 함
    - doc3 에 대한 단어 집합을 기준으로 doc1, doc2의 BoW

In [22]:
vocab_doc1 = [0 for _ in range(len(vocab))]
vocab_doc2 = [0 for _ in range(len(vocab))]

for t1 in okt.morphs(doc1):
    if t1 in vocab.keys():
        vocab_doc1[vocab.get(t1)] +=1

for t2 in okt.morphs(doc2):
    if t2 in vocab.keys():
        vocab_doc2[vocab.get(t2)] +=1
        

print(f" doc3 단어 집합에 대한 doc1 의 Bow : \n {vocab_doc1}")
print(f" doc3 단어 집합에 대한 doc2 의 Bow : \n {vocab_doc2}")

 doc3 단어 집합에 대한 doc1 의 Bow : 
 [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 doc3 단어 집합에 대한 doc2 의 Bow : 
 [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1]


    - 예를 들어 '물가상승률' 은 doc3에서 index 4에 해당하는데
    doc1에서는 2회 등장, doc2 에서는 1회 등장했다.
    
    - BoW는 각 단어가 등장한 횟수를 수치화하는 텍스트 표현 방법으로,
    주로 어떤 단어가 얼마나 등장했는지를 기준으로 문서가 어떤 성격의 문서인지 판단하는 작업에 쓰인다.
    - 즉 분류 문제나 여러 문서 간의 유사도를 구하는 문제에 주로 쓰인다.
    
    예를 들어 '달리기', '체력', '근력'의 단어가 자주 등장하면 해당 문서를 '체육' 관련 문서로, '미분', '방정식',' 부등식' 과 같은 단어가 자주 등장하면 '수학' 관련 문서로 분류할 수 있다.

## CountVectorizer 클래스로 BoW 만들기

    - sklearn에서 단어의 빈도를 count 해서 vector로 만드는 CountVectorizer 클래스 지원

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you']
vector = CountVectorizer()

print(f" bow vector -> {vector.fit_transform(corpus).toarray()}")
print(f" vocabulary -> {dict(sorted(vector.vocabulary_.items(), key = lambda x: x[1]))}")

 bow vector -> [[1 1 2 1 2 1]]
 vocabulary -> {'because': 0, 'know': 1, 'love': 2, 'want': 3, 'you': 4, 'your': 5}


    -> CountVectorizer는 기본적으로 길이가 2이상인 문자에 대해서만 token으로 인식해서, 'I' 사라짐
    CountVectorizer는 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화로 BoW를 만듦
    영어는 띄어쓰기 만으로 토큰화가 수행되도 상관없지만 한국어는 조사 등의 이유로 제대로 BoW가 만들어지기 어려움
    예를 들어 '물가상승률은' 과 '물가상승률과'를 각자 다른 인덱스에서 1이라는 빈도를 갖게 되버림

## 불용어(stop words) 제거 후 BoW 만들기

    - 불용어는 자연어처리에서 별 의미를 갖지 못하는 단어들
    - BoW를 사용한다는 것은 문서내에서 단어가 얼마나 자주 등장했는지를 보고 싶다는 것으로,
    각 단어에 대해 수치화하겠다는 것은 텍스트 내의 어떤 단어들이 중요한지 보고 싶다는 의미가 함축된 것으로
    BoW를 만들 때 불용어를 제거하는 일은 자연어 처리의 정확도를 높일 수 있는 전처리

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

    <사용자가 직접 불용어 넣기>

In [32]:
text = ["Family is not an important thing. It's everything."]
vec = CountVectorizer(stop_words=['the','a','an','is','not'])
print(f" bow : {vec.fit_transform(text).toarray()}")
print(f" vocabulary : {dict(sorted(vec.vocabulary_.items(), key=lambda x: x[1]))}")

 bow : [[1 1 1 1 1]]
 vocabulary : {'everything': 0, 'family': 1, 'important': 2, 'it': 3, 'thing': 4}


    < CountVectorizer에서 지원하는 불용어 사용>

In [34]:
text = ["Family is not an important thing. It's everything."]
vec =  CountVectorizer(stop_words='english')
print(f" bow : {vec.fit_transform(text).toarray()}")
print(f" vocabulary : {dict(sorted(vec.vocabulary_.items(), key=lambda x: x[1]))}")

 bow : [[1 1 1]]
 vocabulary : {'family': 0, 'important': 1, 'thing': 2}


    <NLTK에서 지원하는 불용어 사용>

In [35]:
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words('english')
vec =  CountVectorizer(stop_words=stop_words)
print(f" bow : {vec.fit_transform(text).toarray()}")
print(f" vocabulary : {dict(sorted(vec.vocabulary_.items(), key=lambda x: x[1]))}")

 bow : [[1 1 1 1]]
 vocabulary : {'everything': 0, 'family': 1, 'important': 2, 'thing': 3}
